# Qualidade de dados

Primeiramente os módulos necessários são importados e configurados:

In [1]:
import math as m
import pandas as pd
import statsmodels.imputation.mice

pd.options.display.precision = 2
pd.options.display.max_rows = 6

Além disso, os intervalos de validade são inicializados de acordo com a formulação do problema:

In [2]:
integrity_limits = {
    'A': (10,100),
    'B': (1,2),
    'C': (1,2),
    'D': (1,2),
    'E': (2,7),
    'F': (1,8),
    'G': (1,2),
    'H': (0,7),
    'I': (1,7),
    'J': (1,4),
    'K': (2,8),
    'L': (0,1),
    'M': (0,3),
    'N': (0,3),
    'O': (0,1),
    'P': (0,2),
    'Q': (0,1),
    'R': (0,2),
    'S': (0,4),
    'T': (0,4),
    'U': (0,4),
    'V': (1,9),
    'W': (1,13),
    'X': (1,5),
    'Y': (1,2),
    'Z': (1,2),
}

Por fim, a base de dados é carregada a partir do arquivo de entrada:

In [3]:
df = pd.read_excel('twist_database.xlsx')

Definimos as funções de avaliação para a base de dados. As dimensões analisadas são:

* **Completude**: Avalia a presença do dado. Neste quesito, valores _N/A_ recebem 0. Outros valores recebem 10.
* **Acurácia**: Valores que se encontram dentro do intervalo esperado recebem 10. Caso contrário, recebem 0.
* **Integridade**: Para a base analisada equivale à completude (dados presentes são sempre íntegros).

Como as três dimensões são sempre usadas em conjunto, as funções retornam a nota agregada para cada registro.

In [4]:
def score(x, xmin, xmax):
    """Calcula nota agregada para um único registro"""
    if pd.isna(x):
        comp  = 0
        acc   = 0
        integ = 0
    else:
        comp  = 10
        acc   = 10 if xmin <= x <= xmax else 0
        integ = 10
    
    # Nota agregada é a média das três dimensões
    return (comp + acc + integ) / 3


def column_score(column):
    """Calcula nota agregada para uma coluna"""
    xmin, xmax = integrity_limits[column.name]
    return column.apply(lambda x: score(x, xmin, xmax))


def df_score(df):
    """Calcula nota agregada para um dataframe"""
    return df.apply(column_score)

A partir das funções definidas, podemos calcular as notas para a base de dados. Sobre estas notas são geradas três visões diferentes:

* **Gerencial**: Nota única para a base de dados como um todo. Calculada como a média entre todos os registros da base.
* **Especialista**: Uma nota para cada variável na base. Média entre os registros de uma coluna.
* **Analista**: Uma nota para cada item na base. Média entre os registros de um índice.

In [5]:
scores = df_score(df)
column_mean = scores.mean(0)    # Visão especialista
index_mean  = scores.mean(1)    # Visão analista
total_mean  = index_mean.mean() # Visão gerencial

print('Visão Gerencial:',    total_mean,  sep='\n', end='\n\n')
print('Visão Especialista:', column_mean, sep='\n', end='\n\n')
print('Visão Analista:',     index_mean,  sep='\n', end='\n\n')

Visão Gerencial:
9.05086848635

Visão Especialista:
A    10.00
B    10.00
C     9.84
     ...  
X     8.39
Y     9.52
Z     9.35
Length: 26, dtype: float64

Visão Analista:
0     10.00
1     10.00
2      8.85
      ...  
59    10.00
60    10.00
61    10.00
Length: 62, dtype: float64



Afim de melhor a avaliação da base, pode ser feita a imputação de dados. O código abaixo preenche as lacunas na base a partir dos valores observados. Em seguida, os valores médios de cada variável antes e depois da imputação são listados:

In [9]:
imp = statsmodels.imputation.mice.MICEData(df)
imp.update_all()

df2 = imp.data

print('Médias:')
print('     Antes   Depois')
for c in df.columns:
    mean_before = df[c].mean()
    mean_after = df2[c].mean()
    print('{} {:8.2f} {:8.2f}'.format(c, mean_before, mean_after))

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Médias:
     Antes   Depois
A    48.71    48.71
B     1.16     1.16
C     1.95     1.95
D     1.45     1.45
E     3.29     3.27
F     2.73     2.79
G     1.94     1.94
H     2.27     2.24
I     3.39     3.29
J     2.33     2.37
K     3.93     3.89
L     0.17     0.21
M     0.32     0.27
N     1.13     1.13
O     0.09     0.10
P     0.30     0.32
Q     0.49     0.53
R     0.53     0.53
S     0.72     0.69
T     1.04     1.00
U     1.47     1.45
V     3.24     3.18
W     4.64     4.63
X     1.94     1.95
Y     1.97     1.97
Z     1.97     1.97


Observa-se que o valor médio das variáveis não foi considerávelmente alterado após o procedimento de imputação.